In [7]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from numpy.random import normal
import math
import usefulFunctions
from usefulFunctions import checkForNull
from IPython.core.debugger import Tracer
import random
import numpy
from statsmodels.formula.api import ols
from usefulFunctions import computeMean
from usefulFunctions import computeStdev

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fileName = "Master_PL.xlsx - Master_PL.csv"
table = pd.read_csv(fileName, header=0, 
                    names=["id", "type", "gender", "age", "monthsFromScreening", "protein_BL", "protein_V4",
                          "protein_V6", "protein_V8", "protein_V10", "protein_V12"])


In [55]:
dataWithType = {}
for entry in ["id", "type", "gender", "age", "protein"]:
    dataWithType[entry] = []

dataWithMonthsFromScreening = {}
for entry in ["id", "gender", "age", "monthsFromScreening", "protein"]:
    dataWithMonthsFromScreening[entry] = []

control = {}
affected = {}
male = {}
female = {}
young_age = {}    # younger than 50
middle_1_age = {} # between 50 and 60
middle_2_age = {} # between 60 and 65
middle_3_age = {} # between 65 and 70
old_age = {}      # older than 70

for row in table.to_dict(orient='records'):
    if not checkForNull(row['type']):
        if row['type'] == 1:
            control[int(row['id'])] = row
        else:
            affected[int(row['id'])] = row
    
    if not checkForNull(row['gender']):
        if row['gender'] == 1:
            female[int(row['id'])] = row
        else:
            male[int(row['id'])] = row
    
    if not checkForNull(row['age']):
        if row['age']< 50:
            young_age[int(row['id'])] = row
        elif row['age'] < 60:
            middle_1_age[int(row['id'])] = row
        elif row['age'] < 65:
            middle_2_age[int(row['id'])] = row
        elif row['age'] < 70:
            middle_3_age[int(row['id'])] = row
        else:
            old_age[int(row['id'])] = row
    
    for entry in ["protein_BL", "protein_V4","protein_V6", "protein_V8", "protein_V10", "protein_V12"]:
        if not checkForNull(row[entry]):
            cont1 = True
            cont2 = True
            
            for key in ["id", "type", "gender", "age"]:
                if checkForNull(row[key]):
                    cont1 = False
            
            for key in ["id", "gender", "age", "monthsFromScreening"]:
                if checkForNull(row[key]):
                    cont2 = False
            
            if cont1:
                dataWithType['protein'].append(float(row[entry]))
                for key in ["id", "type", "gender", "age"]:
                    if key == "type" or key == "gender":
                        dataWithType[key].append(row[key]-1)
                    else:
                        dataWithType[key].append(row[key])
            
            if cont2:
                dataWithMonthsFromScreening['protein'].append(float(row[entry]))
                for key in ["id", "gender", "age", "monthsFromScreening"]:
                    if key == "gender":
                        dataWithMonthsFromScreening[key].append(row[key]-1)
                    else:
                        dataWithMonthsFromScreening[key].append(row[key])

In [56]:
dataWithType = pd.DataFrame(dataWithType)
dataWithMonthsFromScreening = pd.DataFrame(dataWithMonthsFromScreening)

In [60]:
dataWithType

,age,gender,id,protein,type
0,32.0,1.0,3011.0,26.0,0.0
1,58.0,1.0,3016.0,66.0,0.0
2,58.0,1.0,3016.0,67.0,0.0
3,58.0,1.0,3016.0,70.0,0.0
4,58.0,1.0,3016.0,56.0,0.0
5,54.0,0.0,3069.0,24.0,0.0
6,54.0,0.0,3069.0,34.0,0.0
7,54.0,0.0,3069.0,32.0,0.0
8,54.0,0.0,3069.0,30.0,0.0
9,54.0,0.0,3069.0,33.0,0.0


In [61]:
 dataWithMonthsFromScreening

,age,gender,id,monthsFromScreening,protein
0,57.0,0.0,3003.0,24.0,39.0
1,57.0,0.0,3003.0,24.0,45.0
2,57.0,0.0,3003.0,24.0,38.0
3,57.0,0.0,3003.0,24.0,29.0
4,58.0,0.0,3006.0,4.0,32.0
5,58.0,0.0,3006.0,4.0,26.0
6,47.0,1.0,3010.0,3.0,74.0
7,47.0,1.0,3010.0,3.0,68.0
8,47.0,1.0,3010.0,3.0,85.0
9,47.0,1.0,3010.0,3.0,134.0


In [63]:
statsForDataWithType = {}
for column in dataWithType:
    statsForDataWithType[column] = {"mean" : computeMean(dataWithType[column]), "stdev" : computeStdev(dataWithType[column])}
    
statsForDataWithMonthsFromScreening = {}
for column in dataWithMonthsFromScreening:
     statsForDataWithMonthsFromScreening[column] = {"mean" : computeMean(dataWithMonthsFromScreening[column]), "stdev" : computeStdev(dataWithMonthsFromScreening[column])}


In [65]:
for i in range(len(dataWithType['age'])):
   dataWithType['age'][i] = (dataWithType['age'][i]-statsForDataWithType['age']['mean'])/statsForDataWithType['age']['stdev']

for i in range(len(dataWithMonthsFromScreening['age'])):
   dataWithMonthsFromScreening['age'][i] = (dataWithMonthsFromScreening['age'][i]-statsForDataWithMonthsFromScreening['age']['mean'])/statsForDataWithMonthsFromScreening['age']['stdev']

for i in range(len(dataWithMonthsFromScreening['monthsFromScreening'])):
     dataWithMonthsFromScreening['monthsFromScreening'][i] = (dataWithMonthsFromScreening['monthsFromScreening'][i]-statsForDataWithMonthsFromScreening['monthsFromScreening']['mean'])/statsForDataWithMonthsFromScreening['monthsFromScreening']['stdev']
    

In [66]:
modelWithType = ols('protein ~ type + gender + age', dataWithType).fit()

In [67]:
print(modelWithType.summary()) 

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     56.17
Date:                Tue, 25 Jul 2017   Prob (F-statistic):           5.87e-35
Time:                        10:02:49   Log-Likelihood:                -9844.2
No. Observations:                2162   AIC:                         1.970e+04
Df Residuals:                    2158   BIC:                         1.972e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    158.1850     30.457      5.194      0.0

In [68]:
modelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', dataWithMonthsFromScreening).fit()

In [69]:
print(modelWithMonthsFromScreening.summary()) 

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     35.01
Date:                Tue, 25 Jul 2017   Prob (F-statistic):           8.42e-22
Time:                        10:04:11   Log-Likelihood:                -6350.9
No. Observations:                1378   AIC:                         1.271e+04
Df Residuals:                    1374   BIC:                         1.273e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              36.6764    